# Machine Learning Engineer Nanodegree
## Capstone Project
## Di-Tech Challenge: Forecasting Supply and Demand Gap for Didi

# Part I - Data Processing

First, let's make sure to import the libraries we need.

In [7]:
import os
import glob
from datetime import datetime, timedelta, time, date
import re
from IPython import display
import matplotlib.pyplot as plt
%matplotlib
import seaborn as sns

import pandas as pd
import numpy as np

Using matplotlib backend: MacOSX


## Data Files Overview
The 600MB data files `training_set.tar.gz` and `test_set.tar.gz` can be downloaded at the [challenge page](http://research.xiaojukeji.com/competition/main.action?competitionId=DiTech2016)<sup>[1]</sup>.<br>
After decompressing,<br> 
`/training_data/` folder contains the following directories and files (ignore the files being recognized as Unix executables):
- `cluster_map` <img alt='cluster_map' src='../src/training_cluster_map.png' width='300'>
- `poi_data` <img alt='poi_data' src='../src/training_poi_data.png' width='300'>
- `order_data` <img alt='order_data' src='../src/training_order_data.png' width='300'>
- `traffic_data` <img alt='traffic_data' src='../src/training_traffic_data.png' width='300'>
- `weather_data` <img alt='weather_data' src='../src/training_weather_data.png' width='300'>

`/test_set_2/` folder contains the following directories and files (ignore the files being recognized as Unix executables):
<img alt='test_set_2' src='../src/test_set_2.png' width='300'>

The target time slots for which gap values need to be predicted are provided in the `read_me_2.txt` in the format of `YYYY-MM-DD-time slot`.
<img alt='read_me_2' src='../src/read_me_2.png' width='150'>

The goal for the data processing is to load and combine all the data files to create a master dataset ready for model building.

[1] _Update_: The current data files on the competition page are for final round. Due to the large size, if you want the first round data files we are using in this project, please contact me and I will send it to you.

## Cluster Map

In [2]:
# load cluaster map data

cluster_map_file = "../raw_data/season_2/training_data/cluster_map/cluster_map"
cluster_map = pd.read_csv(cluster_map_file, delim_whitespace=True, header=None)
cluster_map.columns = ['district_hash', 'district_id']

print 'Dimension of Cluster Map: {}'.format(cluster_map.shape)

Dimension of Cluster Map: (66, 2)


### POI Data

In [3]:
# load poi data

poi_data_file = "../raw_data/season_2/training_data/poi_data/poi_data"
poi_data = pd.DataFrame(index=range(66), columns=['district_hash'] + ['poi_class' + str(i) for i in range(1, 26)])
counter = 0

with open(poi_data_file) as f:
    while True:
        poi_dict = {}
        line = f.readline()
        if line == '':
            break
        data = line.strip().split('\t')
        poi_dist_hash = data[0]
        poi_dict['district_hash'] = poi_dist_hash
        poi_log = data[1:]
        for entry in poi_log:
            entry = re.split(r'\W', entry)
            poi_class = entry[0]
            poi_number = int(entry[-1])
            key = 'poi_class' + poi_class
            if key in poi_dict:
                poi_dict[key] += poi_number
            else:
                poi_dict[key] = poi_number

        poi_data.loc[counter] = pd.Series(poi_dict)
        counter += 1

poi_data.fillna(0, inplace=True)
poi_data = pd.merge(poi_data, cluster_map, how='left', on='district_hash')
poi_data.drop('district_hash', axis=1, inplace=True)
poi_data.set_index('district_id', drop=False, inplace=True)
poi_data.sort_values('district_id', inplace=True)

print 'Dimension of POI data: {}'.format(poi_data.shape)

Dimension of POI data: (66, 26)


### Traffic Data

In [4]:
def load_traffic_data(traffic_data_path):
# load and preprocess traffic data

    all_traffic_data_files = glob.glob(os.path.join(traffic_data_path, "traffic_data*"))
    traffic_data = pd.DataFrame()
    
    for f in all_traffic_data_files:
        df = pd.read_csv(f, delim_whitespace=True, header=None, names=['district_hash', 'level1_tj', 
                                                                       'level2_tj', 'level3_tj', 'level4_tj', 
                                                                       'date', 'timestamp'])

        # create a new variable 'tj_level' = weighted sum of all levels of traffic
        df['tj_level'] = (df['level1_tj'].apply(lambda x: int(x.split(':')[1]) * 0.1) + \
        df['level2_tj'].apply(lambda x: int(x.split(':')[1]) * 0.2) + \
        df['level3_tj'].apply(lambda x: int(x.split(':')[1]) * 0.3) + \
        df['level4_tj'].apply(lambda x: int(x.split(':')[1]) * 0.4)).map(int)

        # convert date to weekday
        df['weekday'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())

        # convert timestamp to time slot 
        df_timestamp = df['timestamp'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))
        df['time'] = df_timestamp.apply(lambda x: int((x - datetime(1900, 1, 1, 0, 0, 0)).total_seconds() 
                                                      / 60.0 / 10 + 1 ))

        # create a new variable datetime that contains date and time in datetime type
        df['datetime'] = df['date'] + ' ' + df['time'].apply(lambda x: str(timedelta(seconds=((int(x) - 1) * 600))))
        df['datetime'] = df['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

        # convert district hash to district ID
        df = df.merge(cluster_map, how='left', on='district_hash')

        # create a new variable 'district_datetime' that combines each individual date time and district
        df['district_datetime'] = df['district_id'].map(str) + ',' + df['date'] + '-' + df['time'].map(str)

        # keep relevant columns
        df.drop(['level1_tj', 'level2_tj', 'level3_tj', 'level4_tj', 'timestamp', 'district_hash'], 
                axis=1, inplace=True)

        traffic_data = traffic_data.append(df, ignore_index=True)
        traffic_data.sort_values(['district_id', 'datetime'], inplace=True)
        traffic_data.set_index(traffic_data['district_datetime'], drop=False, inplace=True)

    return traffic_data

# load training and test traffic data
training_traffic_data_path = "../raw_data/season_2/training_data/traffic_data"
test_traffic_data_path = "../raw_data/season_2/test_set_2/traffic_data"

training_traffic_data = load_traffic_data(training_traffic_data_path)
test_traffic_data = load_traffic_data(test_traffic_data_path)

# write out csv file
training_traffic_data.to_csv('../clean_data/training_traffic_data.csv', index=False)
test_traffic_data.to_csv('../clean_data/test_traffic_data.csv', index=False)

In [9]:
def get_prev_tj(traffic_data):
    # create new variables tj_prev1, tj_prev2 and tj_prev3 that record the tj_level value of last 1st, 2nd and
    # 3rd time slot

    traffic_data.sort_values(['district_id', 'datetime'], inplace=True)

    traffic_data['tj_prev1'] = np.zeros(traffic_data.shape[0])
    traffic_data['tj_prev2'] = np.zeros(traffic_data.shape[0])
    traffic_data['tj_prev3'] = np.zeros(traffic_data.shape[0])

    districts = traffic_data['district_id'].unique().tolist()
    for i in districts:
        indices = traffic_data[traffic_data['district_id']==i].index
        tj_curr = traffic_data[traffic_data['district_id']==i]['tj_level'].values.tolist()
        tj_prev1 = [tj_curr[0]] + tj_curr[:-1]

        traffic_data.ix[traffic_data.district_id==i, 'tj_prev1'] = tj_prev1

        tj_prev2 = [tj_prev1[0]] + tj_prev1[:-1]
        traffic_data.ix[traffic_data.district_id==i, 'tj_prev2'] = tj_prev2

        tj_prev3 = [tj_prev2[0]] + tj_prev2[:-1]
        traffic_data.ix[traffic_data.district_id==i, 'tj_prev3'] = tj_prev3

    return traffic_data

training_traffic_data = get_prev_tj(training_traffic_data)
test_traffic_data = get_prev_tj(test_traffic_data)

### Weather Data

In [5]:
def load_weather_data(weather_data_path):
    # load and preprocess weather data
    
    all_weather_data_files = glob.glob(os.path.join(weather_data_path, "weather_data*"))

    weather_data = pd.DataFrame()

    for f in all_weather_data_files:
        df = pd.read_csv(f, delim_whitespace=True, header=None, names = ['date', 'timestamp', 'weather', 'temp', 'pm'])

        # convert timestamp to time slot
        df_timestamp = df['timestamp'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))
        df_time_slot = df_timestamp.apply(lambda x: int((x - datetime(1900, 1, 1, 0, 0, 0)).total_seconds() 
                                                        / 60.0 / 10 + 1 ))
        df['time'] = df_time_slot.map(int)

        # create a new variable datetime that contains date and time in datetime type
        df['datetime'] = df['date'] + ' ' + df['time'].apply(lambda x: str(timedelta(seconds=((int(x) - 1) * 600))))
        df['datetime'] = df['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

        # remove duplicated measurements for a time slot
        df.drop_duplicates(subset='time', inplace=True)
        
        # keep relevant columns
        df.drop('timestamp', axis=1, inplace=True)
        
        weather_data = weather_data.append(df,ignore_index=True)
        weather_data.sort_values('datetime', inplace=True)
        weather_data.set_index(weather_data['datetime'], drop=False, inplace=True)
        

    return weather_data


# load training and test weather data
training_weather_data_path = "../raw_data/season_2/training_data/weather_data"
test_weather_data_path = "../raw_data/season_2/test_set_2/weather_data"

training_weather_data = load_weather_data(training_weather_data_path)
test_weather_data = load_weather_data(test_weather_data_path)

# write out csv file
training_weather_data.to_csv('../clean_data/training_weather_data.csv', index=False)
test_weather_data.to_csv('../clean_data/test_weather_data.csv', index=False)

### Order Data

In [6]:
# laod order data, takes approximately 10 minutes

def load_order_data(order_data_path):
    # load order data
    
    all_order_data_files = glob.glob(os.path.join(order_data_path, "order_data*"))
    order_data = pd.DataFrame()

    for f in all_order_data_files:
        df = pd.read_csv(f, delim_whitespace=True, header=None, names=['order_id', 'driver_id', 'passenger_id', 
                                                                       'district_hash', 'dest_district_hash',
                                                                       'price', 'date', 'timestamp'])
        # remove duplicated orders
        df.drop_duplicates(subset='order_id', inplace=True)
        
        df.drop(['order_id', 'passenger_id', 'dest_district_hash', 'price'], axis=1, inplace=True)
        
        # convert date to weekday
        df['weekday'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())

        # convert timestamp to time slot 
        df_timestamp = df['timestamp'].apply(lambda x: datetime.strptime(x, '%H:%M:%S'))
        df['time'] = df_timestamp.apply(lambda x: int((x - datetime(1900, 1, 1, 0, 0, 0)).total_seconds() 
                                                      / 60.0 / 10 + 1 ))

        # create a new variable datetime that contains date and time in datetime type
        df['datetime'] = df['date'] + ' ' + df['time'].apply(lambda x: str(timedelta(seconds=((int(x) - 1) * 600))))
        df['datetime'] = df['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

        # convert district hash to district ID
        df = df.merge(cluster_map, how='left', on='district_hash')

        # create a new variable 'district_datetime' that combines each individual date time and district
        df['district_datetime'] = df['district_id'].map(str) + ',' + df['date'] + '-' + df['time'].map(str)


        # keep relevant columns
        df.drop(['district_hash', 'timestamp'], axis=1, inplace=True)

        order_data = order_data.append(df, ignore_index=True)
        order_data.sort_values(['district_id', 'datetime'], inplace=True)
        order_data.set_index(order_data['district_datetime'], drop=False, inplace=True)

    return order_data


# load training and test weather data
training_order_data_path = "../raw_data/season_2/training_data/order_data"
test_order_data_path = "../raw_data/season_2/test_set_2/order_data"

training_order_data = load_order_data(training_order_data_path)
test_order_data = load_order_data(test_order_data_path)

# write out csv file
training_order_data.to_csv('../clean_data/training_order_data.csv', index=False)
test_order_data.to_csv('../clean_data/test_order_data.csv', index=False)

In [10]:
def get_gap(order_data):
    # count NaN driver_id's
    
    gap = pd.DataFrame(order_data.groupby(['district_datetime'])['driver_id'].apply(lambda x: x.isnull().sum()))
    gap.columns = ['gap']
    gap['district_datetime'] = gap.index
    return gap

training_gap = get_gap(training_order_data)
test_gap = get_gap(test_order_data)

In [11]:
def get_all_data(poi_data, traffic_data, weather_data, gap):
    # merge all datasets
    
    all_data = pd.DataFrame(columns=['district_datetime', 'district_id', 'datetime', 
                                     'date', 'time', 'weekday'])
    all_data['district_datetime'] = gap['district_datetime']
    all_data = all_data.merge(gap, how='outer', on='district_datetime')
    all_data['district_id'] = all_data['district_datetime'].apply(lambda x: int(x.split(',')[0]))
    all_data['datetime'] = all_data['district_datetime'].apply(lambda x: x.split(',')[1])
    all_data['date'] = all_data['datetime'].apply(lambda x: '-'.join(x.split('-')[0:3]))
    all_data['weekday'] = all_data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())
    all_data['time'] = all_data['datetime'].apply(lambda x: int(x.split('-')[-1]))
    all_data['datetime'] = all_data['date'] + ' ' + all_data['time'].apply(
        lambda x: str(timedelta(seconds=((int(x) - 1) * 600))))
    all_data['datetime'] = all_data['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

    all_data.sort_values(['district_id', 'datetime'], inplace=True)

    # create new variables gap_prev1, gap_prev2 and gap_prev3 that record the gap value of last 1st, 2nd and
    # 3rd time slot
    all_data['gap_prev1'] = np.zeros(all_data.shape[0])
    all_data['gap_prev2'] = np.zeros(all_data.shape[0])
    all_data['gap_prev3'] = np.zeros(all_data.shape[0])
    
    districts = all_data['district_id'].unique().tolist()
    for i in districts:
        gap_curr = all_data[all_data['district_id']==i]['gap'].values.tolist()
        gap_prev1 = [gap_curr[0]] + gap_curr[:-1]
        all_data.ix[all_data.district_id==i, 'gap_prev1'] = gap_prev1

        gap_prev2 = [gap_prev1[0]] + gap_prev1[:-1]
        all_data.ix[all_data.district_id==i, 'gap_prev2'] = gap_prev2

        gap_prev3 = [gap_prev2[0]] + gap_prev2[:-1]
        all_data.ix[all_data.district_id==i, 'gap_prev3'] = gap_prev3
    

    all_data = all_data.merge(poi_data, how='left', on='district_id')
    all_data = all_data.merge(traffic_data[['district_datetime', 'tj_level', 'tj_prev1', 'tj_prev2', 'tj_prev3']], 
                              how='left', on='district_datetime')   
    all_data = all_data.merge(weather_data[['datetime', 'weather', 'temp', 'pm']], how='left', on='datetime')
    
    all_data.drop('date', axis=1, inplace=True)
    all_data.sort_values(['district_id', 'datetime'], inplace=True)
    all_data.set_index('datetime', drop=False, inplace=True)

    # fill in missing weather data with interpolation
    weather_feature_list = ['weather', 'temp', 'pm']
    for feature in weather_feature_list:
        all_data[feature] = all_data[feature].interpolate(method='time').apply(np.round)
    
    # fill in missing traffic jam level
    traffic_feature_list = ['tj_level', 'tj_prev1', 'tj_prev2', 'tj_prev3']
    for feature in traffic_feature_list:
        all_data[feature] = all_data[feature].fillna(method='bfill')
        all_data[feature] = all_data[feature].fillna(method='ffill')
    
    all_data.sort_values(['datetime', 'district_id'], inplace=True)
    all_data.set_index('datetime', drop=False, inplace=True)
    
    return all_data

training_all_data = get_all_data(poi_data, training_traffic_data, training_weather_data, training_gap)
test_all_data = get_all_data(poi_data, test_traffic_data, test_weather_data, test_gap)

# write out csv file
training_all_data.to_csv('../clean_data/training_all_data.csv', index=False)
test_all_data.to_csv('../clean_data/test_all_data.csv', index=False)

### Targets to be predicted

In [12]:
# load targets to be predicted

target = pd.read_csv("../raw_data/season_2/test_set_2/read_me_2.txt", header=None, skiprows=[0], names=['datetime'])
n = target.shape[0]

districts = np.repeat(np.array(range(1, 67)), n)
                 
target = pd.concat([target]*66, ignore_index=True)
target['district_id'] = pd.Series(districts)
target['date'] = target['datetime'].apply(lambda x: '-'.join(x.split('-')[0:3]))
target['weekday'] = target['datetime'].apply(lambda x: datetime.strptime('-'.join(x.split('-')[0:3]), 
                                                                         '%Y-%m-%d').weekday())
target['time'] = target['datetime'].apply(lambda x: x.split('-')[-1])
target['datetime'] = target['date'] + ' ' + target['time'].apply(
    lambda x: str(timedelta(seconds=((int(x) - 1) * 600))))
target['datetime'] = target['datetime'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
target['district_datetime'] = target['district_id'].map(str) + ',' + target['date'] + '-' + target['time'].map(str)

target.to_csv('../clean_data/target.csv', index=False)

In [13]:
# get test set by combining target and test data 

target_set = pd.merge(test_all_data[['district_id', 'datetime', 'district_datetime', 'weekday', 'time', 'weather', 
                                   'temp', 'pm', 'gap', 'gap_prev1', 'gap_prev2', 'gap_prev3', 'tj_level', 'tj_prev1', 
                                   'tj_prev2', 'tj_prev3']], 
                    target.drop('date', axis=1, inplace=False), how='outer', 
                    on=['district_id', 'datetime', 'district_datetime', 'weekday', 'time'])

target_set = target_set.merge(poi_data, how='outer', on='district_id')

target_set.sort_values(['district_id', 'datetime'], inplace=True)
target_set.set_index('datetime', drop=False, inplace=True)

# fill in missing weather data with interpolation
weather_feature_list = ['weather', 'temp', 'pm']
for feature in weather_feature_list:
    target_set[feature] = target_set[feature].interpolate(method='time').apply(np.round)
    
# fill in missing traffic jam level
traffic_feature_list = ['tj_level', 'tj_prev1', 'tj_prev2', 'tj_prev3']

for feature in traffic_feature_list:
    target_set[feature] = target_set[feature].fillna(method='bfill')
    target_set[feature] = target_set[feature].fillna(method='ffill')
    
target_set.to_csv('../clean_data/target_set.csv', index=False)

print ":D DATA ARE READY FOR DATA ANALYSIS AND MODEL BUILDING :D"

:D DATA ARE READY FOR DATA ANALYSIS AND MODEL BUILDING :D
